In [42]:
import pandas as pd
import numpy as np
import re
import torch
import os
from PIL import Image
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from skimage import io
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer #Pretrained model for text embedding
from torchviz import make_dot
from tqdm import tqdm

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
pd.options.mode.chained_assignment = None  # default='warn'

In [43]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [44]:
def clean_text(text):
    """
    Clean text and convert to lowercase
    """
    return re.sub(r'[^\w\s]', '',text).lower()

def Preprocessing(df):
    """
    Wrapper Function to Preprocess the data (replacing NaNs,
    encoding non-numeric columns to numeric values)
    """
    null_cols=df.isnull().sum()*100/len(df)
    null_cols=null_cols[null_cols!=0]
    
    #Replacing NaNs
    for idx,val in null_cols.items():
        if df.dtypes[idx] == object: #replacing with mode if object (i-e string etc)
            df[idx].fillna(df[idx].mode()[0],inplace=True)        
        else: #replacing with average if integer,float etc
            df[idx].fillna(df[idx].mean(),inplace=True)
     
    #Encoding to numeric values
    alpha_cols=df.select_dtypes(include=['object'])
    try:
        df['overall_sentiment']= df['overall_sentiment'].map({'neutral': 0, 'positive': 1,'negative':2})      
    except:
        pass
    
    #Cleaning text
    for i in range(len(df)):
        try:
            df1['text_corrected'][i] = clean_text(df1['text_corrected'][i])
        except:
            pass
def Preprocessingtest(df):
    """
    Wrapper Function to Preprocess the data (replacing NaNs,
    encoding non-numeric columns to numeric values)
    """
    null_cols=df.isnull().sum()*100/len(df)
    null_cols=null_cols[null_cols!=0]
    
    #Replacing NaNs
    for idx,val in null_cols.items():
        if df.dtypes[idx] == object: #replacing with mode if object (i-e string etc)
            df[idx].fillna(df[idx].mode()[0],inplace=True)        
        else: #replacing with average if integer,float etc
            df[idx].fillna(df[idx].mean(),inplace=True)
     
    #Encoding to numeric values
    alpha_cols=df.select_dtypes(include=['object'])
    #Cleaning text
    for i in range(len(df)):
        try:
            df1['text_corrected'][i] = clean_text(df1['text_corrected'][i])
        except:
            pass
        
    
def get_max_length_words(df,col):
    """
    Gets max number of words in DF column, for padding purposes
    """
    try:
        return df[col].str.split("\\s+").str.len().max()
    except:
        RaiseError("Invalid Column passed")
                
def padding(tensor,length):
    """
    Performs padding
    """
    tensor=torch.tensor(tensor) if not torch.is_tensor(tensor) else tensor
    return torch.nn.functional.pad(tensor,(0, length-tensor.size()[0]))

In [45]:
class MemeDataset(Dataset):
    """
    Custom Dataloader for our dataset.
    All credit goes to: https://github.com/aladdinpersson/Machine-Learning-Collection/blame/master/ML/Pytorch/Basics/custom_dataset_txt/loader_customtext.py
    """
    def __init__(self, dataframe, root_dir, pad_length,model,transform = None, freq_threshold = 5):
        self.df = dataframe.copy()
        self.img = self.df['image_name']
        self.pad_len=pad_length
        self.captions = self.df['text_corrected']
        self.root_dir = root_dir
        self.transform = transform
        self.model=model

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        #convert image to 3 channel RGB, as some images in our dataset are >3 channels or <3 channels
        image = Image.open(os.path.join(self.root_dir, self.df.iloc[index, 0])).convert("RGB")
        y_label = torch.tensor(int(self.df.iloc[index, 2]))
        caption = self.captions[index]
        if self.transform:
            image = self.transform(image)   
        padded_text_tensor = padding(self.model.encode(caption,convert_to_tensor=True),self.pad_len)
        return (image, padded_text_tensor, y_label)

In [61]:
class NN_img(nn.Module):
    """
    Network for images
    """
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Flatten(),
            nn.Linear(100*100*3, 1024),
            nn.ReLU(),
            nn.Linear(1024,512),
            nn.ReLU(),
            nn.Linear(512,256),
            nn.ReLU(),
            nn.Linear(256,60),
            nn.Tanh(),
            nn.Linear(60,3)
        )
            
    def forward(self, x):
        return self.network(x)
    
class NN_text(nn.Module):
    """
    Network for text
    """
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
        nn.Flatten(),
        nn.Linear(200, 160),
        nn.ReLU(),
        nn.Linear(160,128),
        nn.ReLU(),
        nn.Linear(128,72),
        nn.Tanh(),
        nn.Linear(72,64),
        nn.Tanh(),
        nn.Linear(64,3)
        )

    def forward(self, x):
        return self.network(x)    
    
def __train__(dataloader, model, loss_fn, optimizer,flag):
    """
    Trains network on trainset
    """
    size = len(dataloader.dataset)
    for batch, (img, txt, y) in enumerate(dataloader):
        img=img.to(device)
        txt=txt.to(device)
        y=y.to(device)
        if (flag==0):
            pred = model(img)
        else:
            pred = model(txt)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss, current = loss.item(), batch*len(img)
        print(f"loss: {loss:>2f} [{current:>5d}/{size:>2d}]")


def __valid__(dataloader, model, loss_fn,acc_list,flag):
    """
    Performs evaluation on testset
    """
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for img, txt, y in dataloader:
            img=img.to(device)
            txt=txt.to(device)
            y=y.to(device)
            if (flag==0):
                pred = model(img)
            else:
                pred = model(txt)
            y_hat = pred.argmax(dim=1).cpu().numpy()
            pred_txt = [str(w) for w in y_hat]
            print(pred_txt)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss/=num_batches
    correct/=size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.2f}%, Average loss: {test_loss:>2f} \n")
    acc_list.append(100*correct)
def __test__(dataloader, model,label,flag):
    """
    Performs evaluation on testset
    """
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    res=[]
    number=[]

    with torch.no_grad():
        for img, txt, y in dataloader:
            img=img.to(device)
            txt=txt.to(device)
            y=y.to(device)
            if (flag==0):
                pred = model(img)
            else:
                pred = model(txt)
            y_hat = pred.argmax(dim=1).cpu().numpy()
            pred_txt = [str(w) for w in y_hat]
            print(pred_txt)
            print(y.cpu().numpy())
            res.append(pred_txt)
            number.append(y.cpu().numpy())
    return res,number

In [62]:
# 导入os模块
import os
from pandas import DataFrame
path = "C:\\Users\\86187\\AI\\finalproject\\data"
data={'image_name':[],
     'text_corrected':[],
     'overall_sentiment':[]}
number=[]
sentiment=[]
# os.listdir()方法获取文件夹名字，返回数组
with open("C:\\Users\\86187\\AI\\finalproject\\train.txt",'r') as f:
        for line in f:
            s=str(line).strip('\n').split(',')
            number.append(s[0])
            sentiment.append(s[1])
        f.close()
del(number[0])
del(sentiment[0])
file_name_list = os.listdir(path)
img=[]
txt=[]
for i in range(len(file_name_list)):
    if(i%2==0):
        img.append(file_name_list[i])
    else:
        txt.append(file_name_list[i])
for i in range(len(img)):
    z=str(img[i]).split('.')
    if z[0] in number:
        data['image_name'].append(file_name_list[2*i])
        with open("C:\\Users\\86187\\AI\\finalproject\\data\\"+str(txt[i]),'r',encoding="ANSI") as f:
            for line in f:
                data['text_corrected'].append(line)
            f.close()
            data['overall_sentiment'].append(sentiment[number.index(z[0])])
df1=pd.DataFrame(data)
df1

,image_name,text_corrected,overall_sentiment
0,1.jpg,How I feel today #legday #jelly #aching #gym \n,positive
1,10.jpg,@ArrivaTW absolute disgrace two carriages from...,negative
2,100.jpg,This is my Valentine's from 1 of my nephews. I...,positive
3,1001.jpg,Zoe's first love #Rattled @JohnnyHarper15 \n,positive
4,1002.jpg,Chaotic Love - giclee print ?65 at #art #love ...,positive
...,...,...,...
3995,995.jpg,OMG. Well done #Eskom! 'Man dies during #LoadS...,positive
3996,996.jpg,Feelin' the love in here! #ValentinesDay #cari...,positive
3997,997.jpg,#blue #eyes can't be #beaten \n,positive
3998,998.jpg,LA CHUCHA LOUUU TE CHUPO LOS OJOS..! \n,positive


In [63]:
data1={'image_name':[],
     'text_corrected':[],
     'overall_sentiment':[]}
number1=[]
sentiment1=[]
with open("C:\\Users\\86187\\AI\\finalproject\\test_without_label.txt",'r') as f:
        for line in f:
            s=str(line).strip('\n').split(',')
            number1.append(s[0])
            sentiment1.append(s[1])
        f.close()
del(number1[0])
del(sentiment1[0])
file_name_list = os.listdir(path)
img1=[]
txt1=[]
label=[]
for i in range(len(file_name_list)):
    if(i%2==0):
        img1.append(file_name_list[i])
    else:
        txt1.append(file_name_list[i])
for i in range(len(img1)):
    z=str(img1[i]).split('.')    
    if z[0] in number1:
        data1['image_name'].append(file_name_list[2*i])
        with open("C:\\Users\\86187\\AI\\finalproject\\data\\"+str(txt1[i]),'r',encoding="ANSI") as f:
            for line in f:
                data1['text_corrected'].append(line)
            f.close()
            data1['overall_sentiment'].append(z[0])
            label.append(z[0])
df2=pd.DataFrame(data1)
df2

,image_name,text_corrected,overall_sentiment
0,1000.jpg,betterfeelingfilms: RT via Instagram: First da...,1000
1,1020.jpg,RT @GooleAFC: Tonight we recieved a donation o...,1020
2,1021.jpg,#SPEECHLESS. Deah & Yusor were just married in...,1021
3,1025.jpg,RT @AstroTerry: #speechless from this #sunrise \n,1025
4,1036.jpg,How I feel after I #lift! The #pump is #strong...,1036
...,...,...,...
506,948.jpg,#vsco #vscocam #vscohub #vscobest #vscoonly #v...,948
507,96.jpg,Picture Frame #Distressed Red Black #Sentiment...,96
508,960.jpg,RT @FriendsElderly: Loneliness is as big a ris...,960
509,964.jpg,Is the suspense killing you? Cause I'm pretty ...,964


In [64]:
Preprocessing(df1) #Cleaning NaNs, text, encoding sentiment to numeric values
Preprocessingtest(df2)
df1
df2

,image_name,text_corrected,overall_sentiment
0,1000.jpg,betterfeelingfilms: RT via Instagram: First da...,1000
1,1020.jpg,RT @GooleAFC: Tonight we recieved a donation o...,1020
2,1021.jpg,#SPEECHLESS. Deah & Yusor were just married in...,1021
3,1025.jpg,RT @AstroTerry: #speechless from this #sunrise \n,1025
4,1036.jpg,How I feel after I #lift! The #pump is #strong...,1036
...,...,...,...
506,948.jpg,#vsco #vscocam #vscohub #vscobest #vscoonly #v...,948
507,96.jpg,Picture Frame #Distressed Red Black #Sentiment...,96
508,960.jpg,RT @FriendsElderly: Loneliness is as big a ris...,960
509,964.jpg,Is the suspense killing you? Cause I'm pretty ...,964


In [65]:
alpha_len=170 #arbitrary value to add for padding on top of max length sentence
alpha_len1=173
pad_len=get_max_length_words(df1,'text_corrected')+alpha_len
pad_len1=get_max_length_words(df2,'text_corrected')+alpha_len1
df1

,image_name,text_corrected,overall_sentiment
0,1.jpg,how i feel today legday jelly aching gym \n,1
1,10.jpg,arrivatw absolute disgrace two carriages from ...,2
2,100.jpg,this is my valentines from 1 of my nephews i a...,1
3,1001.jpg,zoes first love rattled johnnyharper15 \n,1
4,1002.jpg,chaotic love giclee print 65 at art love chao...,1
...,...,...,...
3995,995.jpg,omg well done eskom man dies during loadsheddi...,1
3996,996.jpg,feelin the love in here valentinesday caring \n,1
3997,997.jpg,blue eyes cant be beaten \n,1
3998,998.jpg,la chucha louuu te chupo los ojos \n,1


In [66]:
vector_model=SentenceTransformer('all-MiniLM-L6-v2').to(device) #Pretrained model to generate text embeddings
#All credit to: https://www.sbert.net/docs/pretrained_models.html
dataset = MemeDataset(dataframe = df1, root_dir = './data', pad_length=pad_len,model=vector_model,transform = transforms.Compose(
    [
     transforms.Resize((100, 100)),
     transforms.ToTensor(),
    transforms.Normalize(torch.Tensor([0.5, 0.5, 0.5]), torch.Tensor([0.5, 0.5, 0.5]))
    ]))
test_set = MemeDataset(dataframe = df2, root_dir = './data', pad_length=pad_len1,model=vector_model,transform = transforms.Compose(
    [
     transforms.Resize((100, 100)),
     transforms.ToTensor(),
    transforms.Normalize(torch.Tensor([0.5, 0.5, 0.5]), torch.Tensor([0.5, 0.5, 0.5]))
    ]))

In [67]:
#Loading train and test sets
train_set, valid_set = torch.utils.data.random_split(dataset, [3000, 1000])
dataloader_train = DataLoader(dataset = train_set, batch_size = 32, shuffle=True)
dataloader_valid = DataLoader(dataset = valid_set, batch_size = 32, shuffle=True)
dataloader_test = DataLoader(dataset = test_set, batch_size = 32, shuffle=True)

In [68]:
#Setting up Neural Networks
net_img = NN_img().to(device) #image network
net_text = NN_text().to(device) #text network 

In [69]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(combined_model.parameters(), lr=0.001)

epochs = 10
acc=list()
for t in tqdm(range(epochs)):
    print(f"Epoch {t+1}\n_______________________________")
    __train__(dataloader_train, net_img, loss_fn, optimizer,0)
    __valid__(dataloader_valid, net_img, loss_fn,acc,0)
    if(t>=2 and t%2==0):
            print(f"Accuracy of t-3 iter: {acc[t-2]}\nAccuracy of current iter: {acc[t]}")
print(f"Highest accuracy was: {max(acc)}")
result=[]
numberlabel=[]
result,numberlabel=__test__(dataloader_test, net_img,label,0)

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

Epoch 1
_______________________________
loss: 1.085944 [    0/3000]
loss: 1.091448 [   32/3000]
loss: 1.088203 [   64/3000]
loss: 1.083897 [   96/3000]
loss: 1.082788 [  128/3000]
loss: 1.088782 [  160/3000]
loss: 1.090992 [  192/3000]
loss: 1.080050 [  224/3000]
loss: 1.084736 [  256/3000]
loss: 1.081335 [  288/3000]
loss: 1.087175 [  320/3000]
loss: 1.086683 [  352/3000]
loss: 1.084102 [  384/3000]
loss: 1.083759 [  416/3000]
loss: 1.085802 [  448/3000]
loss: 1.082688 [  480/3000]
loss: 1.082872 [  512/3000]
loss: 1.085288 [  544/3000]
loss: 1.095365 [  576/3000]
loss: 1.090683 [  608/3000]
loss: 1.084886 [  640/3000]
loss: 1.080611 [  672/3000]
loss: 1.090009 [  704/3000]
loss: 1.083143 [  736/3000]
loss: 1.085111 [  768/3000]
loss: 1.081745 [  800/3000]
loss: 1.086202 [  832/3000]
loss: 1.081379 [  864/3000]
loss: 1.079515 [  896/3000]
loss: 1.089587 [  928/3000]
loss: 1.087119 [  960/3000]
loss: 1.085910 [  992/3000]
loss: 1.087655 [ 1024/3000]
loss: 1.084340 [ 1056/3000]
loss: 1.

 10%|████████▎                                                                          | 1/10 [00:58<08:43, 58.15s/it]

['1', '1', '1', '1', '2', '1', '1', '1', '2', '2', '2', '1', '2', '1', '1', '1', '1', '2', '1', '1', '2', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '2']
['2', '1', '1', '1', '1', '1', '2', '1']
Test Error: 
 Accuracy: 51.00%, Average loss: 1.086283 

Epoch 2
_______________________________
loss: 1.088357 [    0/3000]
loss: 1.084353 [   32/3000]
loss: 1.081016 [   64/3000]
loss: 1.083373 [   96/3000]
loss: 1.084814 [  128/3000]
loss: 1.089684 [  160/3000]
loss: 1.087077 [  192/3000]
loss: 1.089685 [  224/3000]
loss: 1.087618 [  256/3000]
loss: 1.084164 [  288/3000]
loss: 1.089370 [  320/3000]
loss: 1.083380 [  352/3000]
loss: 1.086260 [  384/3000]
loss: 1.079079 [  416/3000]
loss: 1.086592 [  448/3000]
loss: 1.079158 [  480/3000]
loss: 1.086670 [  512/3000]
loss: 1.092564 [  544/3000]
loss: 1.083066 [  576/3000]
loss: 1.083107 [  608/3000]
loss: 1.087422 [  640/3000]
loss: 1.084193 [  672/3000]
loss: 1.085801 [  704/3000]
loss: 1.093749 [  736/3000]
loss: 1.083290 [  768/3000]
l

 20%|████████████████▌                                                                  | 2/10 [01:56<07:47, 58.50s/it]

...

 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [08:43<00:57, 57.86s/it]

['1', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '2', '1', '1', '1', '2', '1', '1', '1', '2', '1', '1', '1', '2', '1', '1', '1', '2', '1', '1']
['2', '1', '1', '1', '1', '1', '1', '1']
Test Error: 
 Accuracy: 51.00%, Average loss: 1.085954 

Accuracy of t-3 iter: 51.0
Accuracy of current iter: 51.0
Epoch 10
_______________________________
loss: 1.080610 [    0/3000]
loss: 1.083139 [   32/3000]
loss: 1.083819 [   64/3000]
loss: 1.085142 [   96/3000]
loss: 1.085751 [  128/3000]
loss: 1.089305 [  160/3000]
loss: 1.083623 [  192/3000]
loss: 1.082636 [  224/3000]
loss: 1.077517 [  256/3000]
loss: 1.090051 [  288/3000]
loss: 1.088615 [  320/3000]
loss: 1.085464 [  352/3000]
loss: 1.085564 [  384/3000]
loss: 1.083763 [  416/3000]
loss: 1.088680 [  448/3000]
loss: 1.081122 [  480/3000]
loss: 1.085281 [  512/3000]
loss: 1.085104 [  544/3000]
loss: 1.090366 [  576/3000]
loss: 1.088007 [  608/3000]
loss: 1.080899 [  640/3000]
loss: 1.093555 [  672/3000]
loss: 1.077925 [  704/3000

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [09:43<00:00, 58.34s/it]

['1', '2', '1', '1', '1', '1', '1', '1', '2', '1', '2', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '2', '1', '1', '1', '1']
['1', '1', '1', '2', '2', '1', '1', '1']
Test Error: 
 Accuracy: 51.00%, Average loss: 1.086119 

Highest accuracy was: 51.0


['1', '1', '1', '1', '1', '2', '2', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '2']
[2863 1096 2029 3637 2469 4372  911 3909 4306 3975 3348 5027 3889 3638
 3224  698  588 1401 2499 2026 1228 2098 2750 5043 4022 3854  960 3927
 1384 3336 4074   38]
['2', '1', '1', '1', '1', '2', '1', '1', '1', '2', '2', '1', '1', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '2', '1', '1', '1', '1', '1', '1', '1']
[1393  657 1768 3400 1587 4288 3149 1037 2646  749 4087 2572 3570   58
 3306 1791 4925 3511 2606 3505  863 2154 2351  932  112 1508 3296 1576
 1111  443 3271 2737]
['1', '1', '2', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '2', '1', '1', '2', '1', '1', '2', '1', '1', '1', '1']
[ 121 2758 3599  479 4138 5017 2991  755 4376  279 3139  313 3003 4046
 1580  573 3119 4898 1233 3641  460 3181 3774 1660 1025 3745 3891 1046
 1059 1969    5 4038]
['1', '1', '1', '1', '1',

In [70]:
acc=list()
for t in tqdm(range(epochs)):
    print(f"Epoch {t+1}\n_______________________________")
    __train__(dataloader_train, net_text, loss_fn, optimizer,1)
    __valid__(dataloader_valid, net_text, loss_fn,acc,1)
    if(t>=2 and t%2==0):
            print(f"Accuracy of t-3 iter: {acc[t-2]}\nAccuracy of current iter: {acc[t]}")
print(f"Highest accuracy was: {max(acc)}")
result=[]
numberlabel=[]
result,numberlabel=__test__(dataloader_test, net_text,label,1)

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

Epoch 1
_______________________________
loss: 1.131173 [    0/3000]
loss: 1.129789 [   32/3000]
loss: 1.127918 [   64/3000]
loss: 1.137351 [   96/3000]
loss: 1.138478 [  128/3000]
loss: 1.122103 [  160/3000]
loss: 1.133584 [  192/3000]
loss: 1.117179 [  224/3000]
loss: 1.118295 [  256/3000]
loss: 1.130945 [  288/3000]
loss: 1.138569 [  320/3000]
loss: 1.138770 [  352/3000]
loss: 1.133483 [  384/3000]
loss: 1.129372 [  416/3000]
loss: 1.134637 [  448/3000]
loss: 1.127127 [  480/3000]
loss: 1.128625 [  512/3000]
loss: 1.138942 [  544/3000]
loss: 1.126982 [  576/3000]
loss: 1.132918 [  608/3000]
loss: 1.136391 [  640/3000]
loss: 1.136866 [  672/3000]
loss: 1.129760 [  704/3000]
loss: 1.128096 [  736/3000]
loss: 1.133010 [  768/3000]
loss: 1.134647 [  800/3000]
loss: 1.124621 [  832/3000]
loss: 1.125157 [  864/3000]
loss: 1.127746 [  896/3000]
loss: 1.131562 [  928/3000]
loss: 1.134481 [  960/3000]
loss: 1.132118 [  992/3000]
loss: 1.122303 [ 1024/3000]
loss: 1.133673 [ 1056/3000]
loss: 1.

 10%|████████▎                                                                          | 1/10 [01:04<09:39, 64.34s/it]

['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
['0', '0', '0', '0', '0', '0', '0', '0']
Test Error: 
 Accuracy: 11.80%, Average loss: 1.128283 

Epoch 2
_______________________________
loss: 1.118120 [    0/3000]
loss: 1.137411 [   32/3000]
loss: 1.129324 [   64/3000]
loss: 1.120309 [   96/3000]
loss: 1.127009 [  128/3000]
loss: 1.132487 [  160/3000]
loss: 1.124508 [  192/3000]
loss: 1.123673 [  224/3000]
loss: 1.122847 [  256/3000]
loss: 1.134051 [  288/3000]
loss: 1.125543 [  320/3000]
loss: 1.132953 [  352/3000]
loss: 1.129354 [  384/3000]
loss: 1.123125 [  416/3000]
loss: 1.137272 [  448/3000]
loss: 1.129203 [  480/3000]
loss: 1.139875 [  512/3000]
loss: 1.141539 [  544/3000]
loss: 1.142043 [  576/3000]
loss: 1.113816 [  608/3000]
loss: 1.137445 [  640/3000]
loss: 1.136533 [  672/3000]
loss: 1.134241 [  704/3000]
loss: 1.128094 [  736/3000]
loss: 1.136146 [  768/3000]
l

 20%|████████████████▌                                                                  | 2/10 [02:06<08:26, 63.29s/it]

...

 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [09:03<00:59, 59.50s/it]

['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
['0', '0', '0', '0', '0', '0', '0', '0']
Test Error: 
 Accuracy: 11.80%, Average loss: 1.127791 

Accuracy of t-3 iter: 11.799999999999999
Accuracy of current iter: 11.799999999999999
Epoch 10
_______________________________
loss: 1.133450 [    0/3000]
loss: 1.129458 [   32/3000]
loss: 1.130635 [   64/3000]
loss: 1.130610 [   96/3000]
loss: 1.134309 [  128/3000]
loss: 1.120962 [  160/3000]
loss: 1.133941 [  192/3000]
loss: 1.129115 [  224/3000]
loss: 1.138490 [  256/3000]
loss: 1.118708 [  288/3000]
loss: 1.128097 [  320/3000]
loss: 1.130363 [  352/3000]
loss: 1.128375 [  384/3000]
loss: 1.116381 [  416/3000]
loss: 1.121584 [  448/3000]
loss: 1.136028 [  480/3000]
loss: 1.133990 [  512/3000]
loss: 1.127244 [  544/3000]
loss: 1.127941 [  576/3000]
loss: 1.131348 [  608/3000]
loss: 1.127174 [  640/3000]
loss: 1.121884 [  672/3000

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [10:02<00:00, 60.20s/it]

['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
['0', '0', '0', '0', '0', '0', '0', '0']
Test Error: 
 Accuracy: 11.80%, Average loss: 1.127069 

Highest accuracy was: 11.799999999999999


['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
[3998 4064 3336 3641  424 5000 2888 1191  121 3572 5027 4471 2943 2991
 1791 1133 1426  657 3955 5092 4450  124 3774  923 2055 3395 3914 2469
 3119  383  812 2154]
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
[1994 2078 4483 1918 2635 4913 4644 4871 3505 4945 3206 2904  264 1806
 1233   31  600 2099   51 1863  677 1227 4431 1168 1596 3248 2348 3052
 2646 4777 3690 1096]
['0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0']
[2499 1699 4046 2019 3842 1933  575 2721  881  960 2085 1949 4877 2351
 4052 3399 2137 1508  116 3222 2915 4922 1417 2115 3417 3637 1406 3599
 1088  566 4721 2358]
['0', '0', '0', '0', '0',